In [2]:
from __future__ import annotations
from exo import *
from exo.libs.memories import DRAM_STATIC
from exo.platforms.x86 import *
from exo.syntax import *

###Define kernel constants
VEC_W = 16

M_REG_BLK = 6
N_REG_BLK = (4 * VEC_W)

M_L1_FAC = 44
N_L1_FAC = 1

M_L1_BLK = M_REG_BLK * M_L1_FAC
N_L1_BLK = N_REG_BLK * N_L1_FAC
K_L1_BLK = 512

@proc
def SGEMM(M: size, N: size, K: size, A: f32[M, K], B: f32[K, N], C: f32[M, N]):
    assert M >= 1
    assert N >= 1
    assert K >= 1
    assert stride(A, 1) == 1
    assert stride(B, 1) == 1
    assert stride(C, 1) == 1

    for k in par(0, K):
        for i in par(0, M):
            for j in par(0, N):
                C[i, j] += A[i, k]*B[k, j]

In [2]:
print(SGEMM.c_code_str())


#include <stdint.h>
#include <stdbool.h>

// Compiler feature macros adapted from Hedley (public domain)
// https://github.com/nemequ/hedley

#if defined(__has_builtin)
#  define EXO_HAS_BUILTIN(builtin) __has_builtin(builtin)
#else
#  define EXO_HAS_BUILTIN(builtin) (0)
#endif

#if EXO_HAS_BUILTIN(__builtin_assume)
#  define EXO_ASSUME(expr) __builtin_assume(expr)
#elif EXO_HAS_BUILTIN(__builtin_unreachable)
#  define EXO_ASSUME(expr) \
      ((void)((expr) ? 1 : (__builtin_unreachable(), 1)))
#else
#  define EXO_ASSUME(expr) ((void)(expr))
#endif

typedef struct c_code_str_Context { 

} c_code_str_Context;


// SGEMM(
//     M : size,
//     N : size,
//     K : size,
//     A : f32[M,K]  @DRAM,
//     B : f32[K,N]  @DRAM,
//     C : f32[M,N]  @DRAM
// )
void SGEMM( c_code_str_Context *ctxt, int_fast32_t M, int_fast32_t N, int_fast32_t K, float* A, float* B, float* C );


static int _floor_div(int num, int quot) {
  int off = (num>=0)? 0 : quot-1;
  return (num-off)/quot;
}

static 

In [3]:
###Create the microkernel
gemm_microkernel4x4 = (SGEMM
                        .rename('gemm_microkernel')
                        .partial_eval(M_L1_BLK, N_REG_BLK, 4))

print(gemm_microkernel4x4.c_code_str())


#include <stdint.h>
#include <stdbool.h>

// Compiler feature macros adapted from Hedley (public domain)
// https://github.com/nemequ/hedley

#if defined(__has_builtin)
#  define EXO_HAS_BUILTIN(builtin) __has_builtin(builtin)
#else
#  define EXO_HAS_BUILTIN(builtin) (0)
#endif

#if EXO_HAS_BUILTIN(__builtin_assume)
#  define EXO_ASSUME(expr) __builtin_assume(expr)
#elif EXO_HAS_BUILTIN(__builtin_unreachable)
#  define EXO_ASSUME(expr) \
      ((void)((expr) ? 1 : (__builtin_unreachable(), 1)))
#else
#  define EXO_ASSUME(expr) ((void)(expr))
#endif

typedef struct c_code_str_Context { 

} c_code_str_Context;


// gemm_microkernel(
//     A : f32[264,4]  @DRAM,
//     B : f32[4,64]  @DRAM,
//     C : f32[264,64]  @DRAM
// )
void gemm_microkernel( c_code_str_Context *ctxt, float* A, float* B, float* C );


static int _floor_div(int num, int quot) {
  int off = (num>=0)? 0 : quot-1;
  return (num-off)/quot;
}

static int8_t _clamp_32to8(int32_t x) {
  return (x < -128)? -128 : ((x > 127)

In [16]:
###Construct a GEBP operation
GEBP = (
        SGEMM
        .rename("GEBP")
        #Change the limits of the loops to be appropriate sizes
        .partial_eval(M=M_L1_BLK)
        .partial_eval(K=K_L1_BLK)
        #Split loops so it calls appropriately sized blocks of the block of A and the panel of B
        .split("i", M_REG_BLK, ["io", "ii"], tail='cut_and_guard')
        .split("j", N_REG_BLK, ["jo", "ji"], tail='cut_and_guard')
        .simplify()
)
print(GEBP.c_code_str())


#include <stdint.h>
#include <stdbool.h>

// Compiler feature macros adapted from Hedley (public domain)
// https://github.com/nemequ/hedley

#if defined(__has_builtin)
#  define EXO_HAS_BUILTIN(builtin) __has_builtin(builtin)
#else
#  define EXO_HAS_BUILTIN(builtin) (0)
#endif

#if EXO_HAS_BUILTIN(__builtin_assume)
#  define EXO_ASSUME(expr) __builtin_assume(expr)
#elif EXO_HAS_BUILTIN(__builtin_unreachable)
#  define EXO_ASSUME(expr) \
      ((void)((expr) ? 1 : (__builtin_unreachable(), 1)))
#else
#  define EXO_ASSUME(expr) ((void)(expr))
#endif

typedef struct c_code_str_Context { 

} c_code_str_Context;


// GEBP(
//     N : size,
//     A : f32[264,512]  @DRAM,
//     B : f32[512,N]  @DRAM,
//     C : f32[264,N]  @DRAM
// )
void GEBP( c_code_str_Context *ctxt, int_fast32_t N, float* A, float* B, float* C );


static int _floor_div(int num, int quot) {
  int off = (num>=0)? 0 : quot-1;
  return (num-off)/quot;
}

static int8_t _clamp_32to8(int32_t x) {
  return (x < -128)? -128 :

In [18]:
###Construct a GEPP operation
GEPP = (SGEMM 
            .rename("GEPP")
            #Change the limits of the loops to be appropriate sizes
            .partial_eval(M=M_L1_BLK)
            #Split loops so it calls appropriately sized blocks of the block of A and the panel of B
            .split("i", M_REG_BLK, ["io", "ii"], tail='cut_and_guard')
            .split("j", N_REG_BLK, ["jo", "ji"], tail='cut_and_guard')
            #Split the K loop so it tiles A 
            .split("k", M_L1_BLK, ["ko", "ki"], tail='cut_and_guard')
            .simplify())

print(GEPP.c_code_str())


#include <stdint.h>
#include <stdbool.h>

// Compiler feature macros adapted from Hedley (public domain)
// https://github.com/nemequ/hedley

#if defined(__has_builtin)
#  define EXO_HAS_BUILTIN(builtin) __has_builtin(builtin)
#else
#  define EXO_HAS_BUILTIN(builtin) (0)
#endif

#if EXO_HAS_BUILTIN(__builtin_assume)
#  define EXO_ASSUME(expr) __builtin_assume(expr)
#elif EXO_HAS_BUILTIN(__builtin_unreachable)
#  define EXO_ASSUME(expr) \
      ((void)((expr) ? 1 : (__builtin_unreachable(), 1)))
#else
#  define EXO_ASSUME(expr) ((void)(expr))
#endif

typedef struct c_code_str_Context { 

} c_code_str_Context;


// GEPP(
//     N : size,
//     K : size,
//     A : f32[264,K]  @DRAM,
//     B : f32[K,N]  @DRAM,
//     C : f32[264,N]  @DRAM
// )
void GEPP( c_code_str_Context *ctxt, int_fast32_t N, int_fast32_t K, float* A, float* B, float* C );


static int _floor_div(int num, int quot) {
  int off = (num>=0)? 0 : quot-1;
  return (num-off)/quot;
}

static int8_t _clamp_32to8(int32_t x) 